In [1]:
from openai import OpenAI
import subprocess
import json

In [2]:
client = OpenAI(
base_url = 'http://localhost:11434/v1',
api_key='ollama', # required, but unused
)

In [3]:
#function to execute the code
def execute_code(code):
    try:
        #Define the code and execute it
        exec(code)
        return locals() #Return the variables created by the code
    except Exception as e:
        return f"Error: {str(e)}"
    

In [4]:
# Define the function schema for OpenAI's function calling

function_descriptions=[{
    'type': 'function',
    'function': {
        'name': 'execute_code',
        'description': 'Executes python code and returns the result.',
        'parameters': {
            'type': 'object',
            'properties': {
                'code': {
                    'type': 'string',
                    'description': 'The python code to execute',
                },
            },
            'required': ['code'],
        },
    },
},
]

tools_list = ['execute_code']

In [30]:
response = client.chat.completions.create(
    model='qwen2.5:14b-instruct-q5_K_M',
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant that executes Python code. If the query does not require python code execution, you dont have to return your response in python code'},
        {'role': 'user', 'content': 'Write a python code to print the sum and multiplication of 5 and 8'}
         ],
    tools=function_descriptions,
    tool_choice = "auto" #Let the model decide when to call the function
)




In [31]:
print(response.choices[0].message)

ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kik0i74y', function=Function(arguments='{"code":"num1 = 5\\nnum2 = 8\\ntotal_sum = num1 + num2\\ntotal_product = num1 * num2\\nprint(f\'Sum: {total_sum}, Multiplication: {total_product}\')"}', name='execute_code'), type='function')])


In [32]:
# Check if tool_calls is not None and has at least one item
if response.choices and response.choices[0].message.tool_calls:
    if tools_list[0] in response.choices[0].message.tool_calls[0].function.name:
        function_args_str = response.choices[0].message.tool_calls[0].function.arguments
        function_args = json.loads(function_args_str)    
        code_to_execute = function_args['code']
        result = execute_code(code_to_execute)
        # print(result)
    elif response.choices[0].message.tool_calls[0].function.name is None:
        print(response.choices[0].message.content)
else:
    print(response.choices[0].message.content)


Sum: 13, Multiplication: 40
